## Ex06. 프로젝트: 멋진 작사가 만들기

## Step 0. 사용할 라이브러리 불러오기

In [1]:
import os, re
import glob  #glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환한다
import numpy as np
import tensorflow as tf

print(tf.__version__)

2.6.0


## Step 1. 데이터 다운로드

In [2]:
# Cloud Shell에서 실행
!mkdir ~/aiffel/lyricist/models
!ln -s ~/data ~/aiffel/lyricist/data

mkdir: cannot create directory ‘/aiffel/aiffel/lyricist/models’: File exists
ln: failed to create symbolic link '/aiffel/aiffel/lyricist/data/data': Read-only file system


## Step 2. 데이터 읽어오기

#### glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이함
glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장

In [3]:
# os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환합니다. 
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*.txt' 

# txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당
txt_list = glob.glob(txt_file_path) 

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        # read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. 
        # splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw = f.read().splitlines()
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## Step 3. 데이터 정제

preprocess_sentence() 함수를 활용해 데이터를 정제

In [4]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지움
#     2. 특수문자 양쪽에 공백을 삽입
#     3. 여러개의 공백은 하나의 공백으로 바꿈
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿈
#     5. 다시 양쪽 공백을 지움
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 여기에 정제된 문장을 모은다
corpus = []
# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뛴다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

## Step 4. 평가 데이터셋 분리

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수로 훈련 데이터와 평가 데이터를 분리

단어장의 크기는 12,000 이상 으로 설정

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거

너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있으므로 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장은 학습 데이터에서 제외함

총 데이터의 20% 를 평가 데이터셋으로 사용

In [5]:
def tokenize(corpus):
    # 12,000단어에 포함되지 못한 단어는 '<unk>'로 바꿈
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줌
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줌
    # (문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용)
    # 한 문장당 토큰의 개수를 최대 15개로 제한
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus) 


from sklearn.model_selection import train_test_split

# tensor로부터 소스문장및 타겟문장을 생성
src_input = tensor[:, :-1]  # 마지막 토큰 제거
tgt_input = tensor[:, 1:]   # 처음 토큰 제거
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2,
                                                          shuffle=True, random_state=32)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f30901defd0>


#### 대이터셋 객체 생성
tf.data.Dataset.from_tensor_slices() 메소드를 이용해 tf.data.Dataset객체를 생성

https://www.tensorflow.org/api_docs/python/tf/data/Dataset

In [6]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12,000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12,001개
# tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
# tokenize() 함수에서 num_words를 12,000개로 선언했기 때문에, tokenizer.num_words의 값은 12,000
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## Step 5. 인공지능 만들기

모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계

loss에 대하여 https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy 참조

In [7]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옴  
embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기
hidden_size = 1024   # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
# tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문 

# 데이터셋에서 데이터 한 배치만 불러오는 방법
for src_sample, tgt_sample in dataset.take(1): break
# 한 배치만 불러온 데이터를 모델에 넣어본다
lyricist(src_sample) 

# 모델의 구조를 확인
lyricist.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


### 모델 학습

In [8]:
# 손실값(loss)이 최소가 되도록 하는것이 학습의 목표이고 학습을 수행하는 알고리즘을 optimizer라고 함
optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저

# 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, # 기본값은 False. 모델에 의해 생성된 출력값이 정규화되지 않았음을 손실 함수에 알려줌
                      # 즉 softmax함수가 적용되지 않았다는걸 의미 
    reduction='none'  # 기본값은 SUM. 각자 나오는 값의 반환 원할 때 None을 사용
)

# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
lyricist.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정
lyricist.fit(dataset, epochs=10)  # epochs=10으로 val_loss 값 2.2 이하가 목표

Epoch 1/10
686/686 [==============================] - 119s 170ms/step - loss: 3.6142
Epoch 2/10
686/686 [==============================] - 117s 171ms/step - loss: 3.1313
Epoch 3/10
686/686 [==============================] - 117s 170ms/step - loss: 2.9379
Epoch 4/10
686/686 [==============================] - 117s 170ms/step - loss: 2.7872
Epoch 5/10
686/686 [==============================] - 117s 171ms/step - loss: 2.6560
Epoch 6/10
686/686 [==============================] - 117s 171ms/step - loss: 2.5364
Epoch 7/10
686/686 [==============================] - 117s 171ms/step - loss: 2.4244
Epoch 8/10
686/686 [==============================] - 117s 171ms/step - loss: 2.3206
Epoch 9/10
686/686 [==============================] - 117s 171ms/step - loss: 2.2216
Epoch 10/10
686/686 [==============================] - 117s 171ms/step - loss: 2.1262


loss값이 2.1262로 목표를 달성함

## Step 6. 모델 평가

In [9]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    return generated #최종적으로 모델이 생성한 문장을 반환

# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

## Step 7. 소회

모델이 "i love you' 라는 문장을 출력하였다.
의외로 간단히 만든 모델이 좋은 결과를 낸다는것이 신기하다.

자연어처리는 영상처리보다 학습하는데 더 많은 시간이 소요되는것을 알 수 있었다.
실시간으로 기사검색을 하여 의미있는 내용을 추출하고 의사결정에 반영하려면 좀더 개선된 모델과 고사양의 장비가 있어야 할 것으로 생각된다.